In [2]:
import tensortrade.env.default as default
from tensortrade.oms.exchanges import Exchange
from tensortrade.feed import Stream
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.oms.instruments import Instrument
from all_indicators import get_all_stock_indicators
from sklearn.preprocessing import MinMaxScaler
import multiprocessing
import pandas as pd
import numpy as np
pd.options.mode.use_inf_as_na = True
RMS_data = get_all_stock_indicators('RMS.PA')
AIR_data = get_all_stock_indicators('AIR.PA')
common_cols = np.intersect1d(RMS_data.columns, AIR_data.columns)
RMS_data=RMS_data[common_cols]
AIR_data=AIR_data[common_cols]

dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26', 'alpha45', 'alpha85'], dtype='object')
dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26', 'alpha94'], dtype='object')


In [17]:
def separate_render_features(stock_data, ticker):
    #ohlc prices for render
    stock_ohlc = stock_data[['Open','High','Low','Close','Volume']].copy()
    stock_ohlc['date'] = stock_ohlc.index + pd.DateOffset(hours=2)
    stock_ohlc = stock_ohlc.add_prefix(f"{ticker}:")
    
    #all features to train from + minmax scaling
    scaler = MinMaxScaler()
    stock_features = stock_data.copy()
    stock_features = stock_features.add_prefix(f"{ticker}:")
    scaler.fit(stock_features)
    stock_features_scaled = pd.DataFrame(scaler.fit_transform(stock_features), columns = stock_features.columns, index = stock_features.index)
    return stock_ohlc, stock_features_scaled

In [18]:
def get_price_stream(stock_renders, tickers): #list
    stock_price_stream_list = []
    for i in range(len(stock_renders)):
        stock_price_stream_list.append(Stream.source(list(stock_renders[i][f"{tickers[i]}:Close"]), dtype="float").rename(f"EUR-{tickers[i].replace('.PA', '')}"))
    return stock_price_stream_list

In [19]:
rms_render, rms_scaled_features = separate_render_features(RMS_data, 'RMS.PA')
air_render, air_scaled_features = separate_render_features(AIR_data, 'AIR.PA')

In [20]:
price_streams = get_price_stream([rms_render, air_render], ["RMS.PA", "AIR.PA"])

In [71]:
#euronext = Exchange('euronext', service=execute_order)(price_streams[0])

In [21]:
euronext = Exchange('euronext', service=execute_order)(price_streams[0], price_streams[1])

In [ ]:
#rms_price_stream = Stream.source(list(rms_ohlc["RMS.PA:Close"]), dtype="float").rename("EUR-RMS")
#euronext = Exchange('euronext', service=execute_order)(rms_price_stream)

In [8]:
with NameSpace("euronext"):
    features_rms = [
        Stream.source(list(rms_scaled_features[feature]), dtype="float").rename(feature) for feature in rms_scaled_features.columns
    ]

feed_rms = DataFeed(features_rms)

In [22]:
with NameSpace("euronext"):
    features2 = [Stream.source(list(air_scaled_features[feature]), dtype="float").rename(feature) for feature in air_scaled_features.columns]
    features3 = [Stream.source(list(rms_scaled_features[feature]), dtype="float").rename(feature) for feature in rms_scaled_features.columns]
feed_rms_air = DataFeed(features2+features3)

In [9]:
feed_rms.compile()

In [23]:
feed_rms_air.compile()

In [25]:
renderer_feed = DataFeed([
    Stream.source(list(rms_render["RMS.PA:date"])).rename("date"),
    Stream.source(list(rms_render["RMS.PA:Open"]), dtype="float").rename("open"),
    Stream.source(list(rms_render["RMS.PA:High"]), dtype="float").rename("high"),
    Stream.source(list(rms_render["RMS.PA:Low"]), dtype="float").rename("low"),
    Stream.source(list(rms_render["RMS.PA:Close"]), dtype="float").rename("close"), 
    Stream.source(list(rms_render["RMS.PA:Volume"]), dtype="float").rename("volume"),
    Stream.source(list(air_render["AIR.PA:date"])).rename("date"),
    Stream.source(list(air_render["AIR.PA:Open"]), dtype="float").rename("open"),
    Stream.source(list(air_render["AIR.PA:High"]), dtype="float").rename("high"),
    Stream.source(list(air_render["AIR.PA:Low"]), dtype="float").rename("low"),
    Stream.source(list(air_render["AIR.PA:Close"]), dtype="float").rename("close"), 
    Stream.source(list(air_render["AIR.PA:Volume"]), dtype="float").rename("volume"),
])

In [26]:
EUR = Instrument('EUR', 4, 'Euro')
RMS = Instrument('RMS', 4, 'Hermes')
AIR = Instrument('AIR', 4, 'Hermes')

cash = Wallet(euronext, 10000 * EUR)
asset_rms = Wallet(euronext, 0 * RMS)
asset_air = Wallet(euronext, 0 * AIR)

portfolio = Portfolio(EUR, [cash, asset_rms, asset_air])

In [27]:
portfolio.wallets

[<Wallet: balance=10000.0000 EUR, locked=0.0000 EUR>,
 <Wallet: balance=0.0000 RMS, locked=0.0000 RMS>,
 <Wallet: balance=0.0000 AIR, locked=0.0000 AIR>]

In [ ]:
from sklearn.model_selection import train_test_split

def split_data(data):
    X = data.copy().drop(columns=['Log Returns','Returns'])
    y = data.copy()['Returns']

    X_train_test, X_valid, y_train_test, y_valid = \
        train_test_split(X, y, train_size=0.9, test_size=0.1, shuffle=False)

    X_train, X_test, y_train, y_test = \
        train_test_split(X_train_test, y_train_test, train_size=0.8, test_size=0.2, shuffle=False)

    return X_train, X_test, X_valid, y_train, y_test, y_valid

In [ ]:
X_train, X_test, X_valid, y_train, y_test, y_valid = split_data(RMS_data)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from feature_engine.selection import SelectBySingleFeaturePerformance

rf = RandomForestClassifier(n_estimators=100, 
                            random_state=1337, 
                            n_jobs=6)

sel = SelectBySingleFeaturePerformance(variables=None, 
                                       estimator=rf, 
                                       scoring="roc_auc", 
                                       cv=5, 
                                       threshold=0.5)

sel.fit(X_train, y_train)

#changer y_train en sortino

In [ ]:
sel.feature_performance_

In [12]:
window_size = 100
#1028
n_steps = 400
def get_optimal_batch_size(window_size=100, n_steps=400, batch_factor=4, stride=1):
    """
    lookback = 30          # Days of past data (also named window_size).
    batch_factor = 4       # batch_size = (sample_size - lookback - stride) // batch_factor
    stride = 1             # Time series shift into the future.
    """
    lookback = window_size
    sample_size = n_steps
    batch_size = ((sample_size - lookback - stride) // batch_factor)
    return batch_size

batch_size = get_optimal_batch_size(window_size=window_size, n_steps=n_steps, batch_factor=4)
batch_size

74

In [28]:
from tensortrade.agents import DQNAgent
from tensortrade.env.default.actions import BSH, ManagedRiskOrders
from tensortrade.env.default.rewards import RiskAdjustedReturns
from tensortrade.env.default.renderers import PlotlyTradingChart
import threading

reward_scheme = RiskAdjustedReturns(return_algorithm='sortino', window_size=100)
action_scheme = ManagedRiskOrders()

chart_renderer = PlotlyTradingChart(
    display=True,  # show the chart on screen (default)
    height=800,  # affects both displayed and saved file height. None for 100% height.
    save_format="html",  # save the chart to an HTML file
    auto_open_html=True,  # open the saved HTML chart in a new browser tab
)

env = default.create(
    portfolio=portfolio,
    action_scheme=action_scheme,
    reward_scheme=reward_scheme,
    feed=feed_rms_air, #feed,
    renderer_feed=renderer_feed,
    renderer=chart_renderer,
    window_size=100
)

 #(1029 days - 4 trading years)
n_episodes = 1
memory_capacity = n_steps * 10
save_path = 'agents/'
n_bins = 5             # Number of bins to partition the dataset evenly in order to evaluate class sparsity.
seed = 1337
commission = 0.001

agent = DQNAgent(env)

agent.train(batch_size=batch_size, 
            n_steps=n_steps, 
            n_episodes=n_episodes, 
            memory_capacity=memory_capacity, 
            save_path=save_path)


====      AGENT ID: 62863b25-7b2c-4b48-8af0-727ff1f0fe1a      ====


  0%|          | 0/1 [00:00<?, ?it/s]

FigureWidget({
    'data': [{'name': 'Price',
              'showlegend': False,
              'type': 'candle…

FigureWidget({
    'data': [{'close': array([ 83.84531403,  82.35800171,  83.40288544,  86.11396027,  84.69252…

FigureWidget({
    'data': [{'close': array([ 83.84531403,  82.35800171,  83.40288544,  86.11396027,  84.69252…

...

FigureWidget({
    'data': [{'close': array([ 83.84531403,  82.35800171,  83.40288544, ..., 128.9886322 ,
    …

FigureWidget({
    'data': [{'close': array([83.84531403, 82.35800171, 83.40288544, ..., 85.9065094 , 85.55330…

FigureWidget({
    'data': [{'close': array([83.84531403, 82.35800171, 83.40288544, ..., 54.9619751 , 55.27745…

100%|██████████| 1/1 [08:39<00:00, 519.19s/it]


0.0021851951121585776

In [ ]:
# Create a list to store the threads
threads = []

# Create a thread for each instance of the DQNAgent
for agent in agents:
    thread = Thread(target=train_agent, args=(agent, batch_size, n_steps, n_episodes, memory_capacity, save_path))
    threads.append(thread)

# Start the threads
for thread in threads:
    thread.start()

In [ ]:
from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2


policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5 }
agent = PPO2(MlpLnLstmPolicy, env)

https://www.tensortrade.org/en/latest/agents/overview.html#stable-baselines

#agent = model(policy, env)#, model_kwargs=params)

In [ ]:
pip install stable-baselines3

In [ ]:
import tensorflow
tensorflow.compat.v1.get_default_session()

In [ ]:
tensorflow.compat.v1.get_default_session()
https://www.tensortrade.org/en/latest/examples/train_and_evaluate_using_ray.htmlhttps://www.tensortrade.org/en/latest/examples/train_and_evaluate_using_ray.htmlhttps://www.tensortrade.org/en/latest/examples/train_and_evaluate_using_ray.htmlhttps://www.tensortrade.org/en/latest/examples/train_and_evaluate_using_ray.html

In [ ]:

print(tensorflow.version.VERSION)

In [ ]:
pip install ray[rllib]

In [ ]:
pd.DataFrame(portfolio.performance).iloc[-2:-1,:].T.plot()

In [ ]:
rms_ohlc['RMS.PA:Close'].iloc[:100].plot()

In [ ]:
AIR['Close'].iloc[:100].plot()

In [ ]:
#https://www.tensortrade.org/en/latest/examples/setup_environment_tutorial.html
#https://github.com/tensortrade-org/tensortrade/blob/master/examples/train_and_evaluate.ipynb
https://levelup.gitconnected.com/portfolio-allocation-with-tensortrade-part-2-2-9ac30a6bcbfe
https://www.tensortrade.org/en/latest/agents/overview.html#stable-baselines
https://levelup.gitconnected.com/portfolio-allocation-with-tensortrade-part-2-2-9ac30a6bcbfe
    https://github.com/Tomas0413/tensortrade-experiments/blob/main/TensorTrade%20-%20Sinewave%20with%20SimpleProfit%20and%20ManagedRiskOrders.ipynb
    https://www.google.com/search?q=feature_engine&sourceid=chrome&ie=UTF-8
        https://github.com/tensortrade-org/tensortrade/blob/master/examples/train_and_evaluate.ipynb